In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import random as rd
from copy import deepcopy
from spp.convex_sets import Singleton, Polyhedron
from spp.convex_functions import TwoNorm, SquaredTwoNorm
from spp.graph import GraphOfConvexSets
from spp.shortest_path import ShortestPathProblem

In [ ]:
def shuffle(l):
    '''
    Shuffles the given list randomly.
    '''
    return rd.sample(list(l), k=len(l))

def split(l, n):
    '''
    Splits the given list in n sublists of random length.
    '''
    assert 1 <= n <= len(l)
    split_if = shuffle([True] * (n - 1) + [False] * (len(l) - n))
    split_at = np.where([True] + split_if + [True])[0]
    return [l[i:j] for i, j in zip(split_at[:-1], split_at[1:])]

def partition(l):
    '''
    Partitions the given list in n sublists of random length.
    n is drawn uniformly at random from [1, len(l)].
    '''
    n = rd.randint(1, len(l))
    return split(l, n)

def paths(V):
    '''
    Generates random paths by partitioning the set V - {s,t}.
    The number of paths (sets in the partition) is random.
    The number of vertices traversed by each path is random too.
    '''
    return [[V[0]] + p + [V[-1]] for p in partition(V[1:-1])]

def edges_in_path(p):
    '''
    Returns the edges in the given path (list of vertices).
    '''
    return list(zip(p[:-1], p[1:]))

def edges_in_paths(ps):
    '''
    Returns all the edges in the given list of paths.
    No edge is repeated.
    '''
    return list(set(e for p in ps for e in edges_in_path(p)))

def extend_edges(E, V, nE):
    '''
    Extends the given edge set until it contains nE edges.
    Does not add self edges and does not generate repetitions.
    '''
    E = deepcopy(E)
    while len(E) < nE:
        i = rd.choice(V)
        j = rd.choice(V)
        e = (i, j)
        if i != j and not e in E:
            E.append(e)
    return E
            
def box(d, vol):
    '''
    Returns an axis-aligned box in R^d of the given volume.
    The center of the box is drawn uniformly at random in [0,1]^d.
    '''
    center = np.array([rd.uniform(0, 1) for i in range(d)])
    side = .5 * vol ** (1 / d) * np.ones(d)
    x_min = center - side
    x_max = center + side
    return Polyhedron.from_bounds(x_min, x_max)
    
def get_graph(params, l=None):
    '''
    Constructs a random instance of the SPP.
    '''
    assert params['nE'] >= params['nV'] - 1
    
    # graph
    G = GraphOfConvexSets()
    
    # convex sets
    Xs = Singleton(np.zeros(params['d']))
    Xt = Singleton(np.ones(params['d']))
    X = [Xs]
    for v in range(params['nV'] - 2):
        Xv = box(params['d'], params['vol'])
        X.append(Xv)
    X.append(Xt)
    V = list(range(params['nV']))
    G.add_sets(X, V)
    G.set_source(0)
    G.set_target(params['nV'] - 1)
    
    # edges
    E = edges_in_paths(paths(V))
    E = extend_edges(E, V, params['nE'])
    for e in E:
        G.add_edge(e[0], e[1], l)
    
    return G

In [ ]:
def solve(params, n_trials):
    
    I = np.eye(params['d'])
    H = np.hstack((I, -I))
    l = {'E': TwoNorm(H), 'E2': SquaredTwoNorm(H)}
    
    cost = {
        0: {'E': np.zeros(n_trials), 'E2': np.zeros(n_trials)},
        1: {'E': np.zeros(n_trials), 'E2': np.zeros(n_trials)}
    }
    time = deepcopy(cost)
    
    for i in range(n_trials):
        G = get_graph(params)
        print(f'Trial {i}', end ='\r')
        for relaxation in [0, 1]:
            for norm in ['E', 'E2']:
                for e in G.edges:
                    G.set_edge_length(e, l[norm])    
                spp = ShortestPathProblem(G, relaxation=relaxation)
                sol = spp.solve()
                cost[relaxation][norm][i] = sol.cost
                time[relaxation][norm][i] = sol.time
                
    return cost, time

def set_stat(stat, values):
    stat['mean'] = np.mean(values)
    stat['median'] = np.median(values)
    stat['max'] = max(values)

def rel_gap_stat(cost):
    stat = {'E': {}, 'E2': {}}
    for norm in ['E', 'E2']:
        gaps = 1 - cost[1][norm] / cost[0][norm]
        set_stat(stat[norm], gaps)
    return stat

def time_stat(time):
    stat = {
        0: {'E': {}, 'E2': {}},
        1: {'E': {}, 'E2': {}}
    }
    for relaxation in [0, 1]:
        for norm in ['E', 'E2']:
            set_stat(stat[relaxation][norm], time[relaxation][norm])
    return stat

tab = lambda n : '    ' * n

def print_rel_gap(stat, label):
    print(label + ', relaxation gap:')
    for norm in ['E', 'E2']:
        print(tab(1), 'Norm:', norm)
        for key, value in stat[norm].items():
            print(tab(2), key, round(value, 4))

def print_time(stat, label):
    print(label + ', time:')
    for relaxation in [0, 1]:
        print(tab(1), 'Relaxation:', relaxation)
        for norm in ['E', 'E2']:
            print(tab(2), 'Norm:', norm)
            for key, value in stat[relaxation][norm].items():
                print(tab(3), key, round(value, 4))

In [ ]:
n_trials = 100
params_nom = {
    'd': 4,
    'nE': 100,
    'nV': 50,
    'vol': .01
}
rd.seed(0)
cost_nom, time_nom = solve(params_nom, n_trials)
print_rel_gap(rel_gap_stat(cost_nom), 'Nominal')
print_time(time_stat(time_nom), 'Nominal')

In [ ]:
params_max = [
    {'d': 20},
    {'nE': 500},
    {'nV': 250, 'nE': 500},
    {'vol': .05}
]
results = {}
for params_change in params_max:
    rd.seed(0)
    params = deepcopy(params_nom)
    for p, v in params_change.items():
        params[p] = v
    cost, time = solve(params, n_trials)
    label = ' '.join(params_change.keys())
    results[label] = [cost, time]
    print_rel_gap(rel_gap_stat(cost), label)
    print_time(time_stat(time), label)
    print('\n')

# Plot 2d projection of 4d instance

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_graph(G):
    plt.gca().set_aspect('equal')

    # sets
    for i, Xi in enumerate(list(G.sets.values())[1:-1]):
        plt.scatter(*Xi.center, s=15, facecolor='k', alpha=.2)
        Xi.plot(alpha=.5, facecolor='mintcream', edgecolor='k', zorder=0)

    # edges
    kw = dict(arrowstyle='->, head_width=3, head_length=8', color='k', linewidth=.2)
    for e in G.edges:
        c0 = G.sets[e[0]].center
        c1 = G.sets[e[1]].center
        plt.gca().add_patch(patches.FancyArrowPatch(c0, c1, **kw))

    # start and goal points
    plt.scatter(0, 0, c='k')
    plt.scatter(1, 1, c='k')
    plt.text(-.05, 0, r'$s$', ha='right', va='center', size=14, zorder=3)
    plt.text(1.05, 1, r'$t$', ha='left', va='center', size=14, zorder=3)
    
rd.seed(0)
params = {
    'd': 2,
    'nE': 100,
    'nV': 50,
    'vol': .01 ** (2/4)
}
G = get_graph(params)
    
plt.figure(figsize=(3.5,3.5))
plt.gca().axis('off')
plot_graph(G)
plt.savefig('random_instance.pdf', bbox_inches='tight')